<div style="background-color:#EAF4EC; padding:20px; border-radius:10px;">

<h2 style="color:#2F6F4E; text-align:center; margin-bottom:5px;">
Forecasting — Dataset & Baselines
</h2>

<h4 style="color:#2F6F4E; text-align:center; margin-top:0;">
Master Thesis – ESG Governance Indicators (EU-27)
</h4>

<p style="font-size:14px; color:#2F6F4E;">
This notebook implements the <strong>Forecasting</strong> stage of the CRISP-ML(Q) methodology.
The objective is to define a rigorous forecasting setup and build <strong>baseline models</strong> for the
EU-27 governance indicators, creating a benchmark for more advanced machine learning approaches.
</p>

<p style="font-size:14px; color:#2F6F4E;">
This notebook focuses on <strong>problem formulation and benchmarking</strong>. No advanced models are trained here.
The outputs generated will serve as the foundation for forecasting model development and evaluation in subsequent notebooks.
</p>

</div>


In [24]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import optuna

In [25]:
DATA_PATH = "../data/processed/data_final_governance_EU27.csv"
df_final = pd.read_csv(DATA_PATH)

print(df_final.shape)
df_final.head(2)


(405, 28)


Country Name Country Code                   Indicator Name  \
0      Austria          AUT  Control of Corruption: Estimate   
1      Austria          AUT            GDP growth (annual %)   

      Indicator Code      2000      2001      2002      2003      2004  \
0             CC.EST  1.751059  1.751059  1.915159  1.963869  2.026624   
1  NY.GDP.MKTP.KD.ZG  3.189523  1.316987  1.484369  1.141565  2.565255   

       2005  ...      2014      2015      2016     2017      2018      2019  \
0  1.912423  ...  1.466502  1.472044  1.496881  1.50082  1.568587  1.521324   
1  2.320365  ...  0.755799  1.303523  2.117220  2.27225  2.484221  1.754976   

       2020      2021      2022      2023  
0  1.477709  1.242727  1.258587  1.133653  
1 -6.318255  4.795329  5.277894 -0.954963  

[2 rows x 28 columns]

<div style="background-color:#EAF4EC; padding:16px; border-radius:10px;">
<h2 style="color:#2F6F4E; margin-bottom:5px; font-size:20px;">
Identification of Temporal Dimensions
</h2>
<p style="color:#2F6F4E; font-size:14px;">
Detection of year-based columns to structure the dataset in a time-aware format suitable for forecasting.
</p>
</div>


In [26]:
# Identify year columns (e.g. 2000, 2001, ..., 2023)
year_cols = [c for c in df_final.columns if c.isdigit()]

print(f"Detected {len(year_cols)} year columns:")
print(year_cols[:5], "...", year_cols[-5:])

Detected 24 year columns:
['2000', '2001', '2002', '2003', '2004'] ... ['2019', '2020', '2021', '2022', '2023']


<div style="background-color:#EAF4EC; padding:16px; border-radius:10px;">
<h2 style="color:#2F6F4E; margin-bottom:5px; font-size:20px;">
Reshaping Data to Long Format
</h2>
<p style="color:#2F6F4E; font-size:14px;">
Conversion from wide to long format to obtain a country–indicator–year structure required for time series modeling.
</p>
</div>


In [27]:
# Melt to long format (country–year–indicator)
df_long = df_final.melt(
    id_vars=["Country Name", "Country Code", "Indicator Name", "Indicator Code"],
    value_vars=year_cols,
    var_name="year",
    value_name="value"
)

df_long["year"] = df_long["year"].astype(int)

print(df_long.shape)
df_long.head(2)

(9720, 6)


Country Name Country Code                   Indicator Name  \
0      Austria          AUT  Control of Corruption: Estimate   
1      Austria          AUT            GDP growth (annual %)   

      Indicator Code  year     value  
0             CC.EST  2000  1.751059  
1  NY.GDP.MKTP.KD.ZG  2000  3.189523

In [28]:
year_cols = [c for c in df_final.columns if c.isdigit()]
year_cols = sorted(year_cols, key=int)

df_long = df_final.melt(
    id_vars=["Country Name", "Country Code", "Indicator Name", "Indicator Code"],
    value_vars=year_cols,
    var_name="year",
    value_name="value"
)

df_long["year"] = df_long["year"].astype(int)
df_long["value"] = pd.to_numeric(df_long["value"], errors="coerce")

df_long = df_long.sort_values(["Country Code", "Indicator Code", "year"]).reset_index(drop=True)
df_long["series_id"] = df_long["Country Code"] + "_" + df_long["Indicator Code"]

assert df_long["value"].isna().sum() == 0
assert df_long.duplicated(subset=["Country Code", "Indicator Code", "year"]).sum() == 0

print(df_long.shape)
df_long.head(2)

(9720, 7)


Country Name Country Code                   Indicator Name Indicator Code  \
0      Austria          AUT  Control of Corruption: Estimate         CC.EST   
1      Austria          AUT  Control of Corruption: Estimate         CC.EST   

   year     value   series_id  
0  2000  1.751059  AUT_CC.EST  
1  2001  1.751059  AUT_CC.EST

Each line represetn 1 country x 1 indicator x 1 year

In [29]:
series_length = (
    df_long
    .groupby("series_id")["year"]
    .nunique()
)

series_length.describe()

count    405.0
mean      24.0
std        0.0
min       24.0
25%       24.0
50%       24.0
75%       24.0
max       24.0
Name: year, dtype: float64

In [30]:
complete_series = series_length[series_length == 24].index
len(complete_series)

405

In [31]:
df_long.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9720 entries, 0 to 9719
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country Name    9720 non-null   object 
 1   Country Code    9720 non-null   object 
 2   Indicator Name  9720 non-null   object 
 3   Indicator Code  9720 non-null   object 
 4   year            9720 non-null   int64  
 5   value           9720 non-null   float64
 6   series_id       9720 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 531.7+ KB


In [32]:
df_series = (
    df_long
    .sort_values(["Country Code", "Indicator Code", "year"])
    .reset_index(drop=True)
)

df_series["series_id"] = (
    df_series["Country Code"] + "_" + df_series["Indicator Code"]
)

<div style="background-color:#EAF4EC; padding:16px; border-radius:10px;">
<h2 style="color:#2F6F4E; margin-bottom:5px; font-size:20px;">
Indicator Selection for Forecasting
</h2>
<p style="color:#2F6F4E; font-size:14px;">
Isolation of a single governance indicator to enable focused and interpretable forecasting experiments.
</p>
</div>


In [33]:
# Select one indicator to start with
indicator_name = "Control of Corruption: Estimate"

df_ind = df_series[df_series["Indicator Name"] == indicator_name].copy()

print(df_ind.shape)
df_ind.head(2)

(648, 7)


Country Name Country Code                   Indicator Name Indicator Code  \
0      Austria          AUT  Control of Corruption: Estimate         CC.EST   
1      Austria          AUT  Control of Corruption: Estimate         CC.EST   

   year     value   series_id  
0  2000  1.751059  AUT_CC.EST  
1  2001  1.751059  AUT_CC.EST

In [34]:
df_ind["Country Name"].nunique(), df_ind["year"].min(), df_ind["year"].max()

(27, np.int64(2000), np.int64(2023))

In [35]:
df_ind = df_ind.sort_values(["Country Name", "year"]).reset_index(drop=True)

<div style="background-color:#EAF4EC; padding:16px; border-radius:10px;">
<h2 style="color:#2F6F4E; margin-bottom:5px; font-size:20px;">
Lag Feature Engineering
</h2>
<p style="color:#2F6F4E; font-size:14px;">
Generation of lagged values to capture short-term temporal dependencies within each country.
</p>
</div>


In [36]:
# Create lag features (per country)
for lag in [1, 2, 3]:
    df_ind[f"lag_{lag}"] = (
        df_ind
        .groupby("Country Name")["value"]
        .shift(lag)
    )

df_ind.head(2)

Country Name Country Code                   Indicator Name Indicator Code  \
0      Austria          AUT  Control of Corruption: Estimate         CC.EST   
1      Austria          AUT  Control of Corruption: Estimate         CC.EST   

   year     value   series_id     lag_1  lag_2  lag_3  
0  2000  1.751059  AUT_CC.EST       NaN    NaN    NaN  
1  2001  1.751059  AUT_CC.EST  1.751059    NaN    NaN

<div style="background-color:#EAF4EC; padding:16px; border-radius:10px;">
<h2 style="color:#2F6F4E; margin-bottom:5px; font-size:20px;">
Rolling Window Statistics
</h2>
<p style="color:#2F6F4E; font-size:14px;">
Computation of rolling mean and volatility measures to smooth noise and represent medium-term trends.
</p>
</div>


In [37]:
# Garantir ordenação temporal correta
df_ind = df_ind.sort_values(["Country Name", "year"]).copy()

g = df_ind.groupby("Country Name")["value"]

# Rolling statistics (window = 3 years) — apenas informação passada
df_ind["rolling_mean_3"] = g.transform(
    lambda s: s.shift(1).rolling(window=3).mean()
)

df_ind["rolling_std_3"] = g.transform(
    lambda s: s.shift(1).rolling(window=3).std()
)

# Trend (short-term momentum)
df_ind["trend"] = df_ind["lag_1"] - df_ind["lag_2"]

df_ind.head(2)

Country Name Country Code                   Indicator Name Indicator Code  \
0      Austria          AUT  Control of Corruption: Estimate         CC.EST   
1      Austria          AUT  Control of Corruption: Estimate         CC.EST   

   year     value   series_id     lag_1  lag_2  lag_3  rolling_mean_3  \
0  2000  1.751059  AUT_CC.EST       NaN    NaN    NaN             NaN   
1  2001  1.751059  AUT_CC.EST  1.751059    NaN    NaN             NaN   

   rolling_std_3  trend  
0            NaN    NaN  
1            NaN    NaN

<div style="background-color:#EAF4EC; padding:16px; border-radius:10px;">
<h2 style="color:#2F6F4E; margin-bottom:5px; font-size:20px;">
Handling Initial Missing Values
</h2>
<p style="font-size:14px; color:#2F6F4E">
Removal of initial observations affected by lag and rolling window construction to ensure model validity.
</p>
</div>


In [38]:
# Drop initial rows with NaN caused by lags/rolling
req = ["value","lag_1","lag_2","lag_3","rolling_mean_3","rolling_std_3","trend"]
df_model = df_ind.dropna(subset=req).reset_index(drop=True)

print(df_model.shape)
df_model.head(2)

(567, 13)


Country Name Country Code                   Indicator Name Indicator Code  \
0      Austria          AUT  Control of Corruption: Estimate         CC.EST   
1      Austria          AUT  Control of Corruption: Estimate         CC.EST   

   year     value   series_id     lag_1     lag_2     lag_3  rolling_mean_3  \
0  2003  1.963869  AUT_CC.EST  1.915159  1.751059  1.751059        1.805759   
1  2004  2.026624  AUT_CC.EST  1.963869  1.915159  1.751059        1.876696   

   rolling_std_3     trend  
0       0.094743  0.164101  
1       0.111497  0.048709

Initial observations lacking sufficient historical context were removed after temporal feature construction to ensure model validity and prevent information leakage.

<div style="background-color:#EAF4EC; padding:16px; border-radius:10px;">
<h2 style="color:#2F6F4E; margin-bottom:5px; font-size:20px;">
Time-Based Dataset Splitting
</h2>
<p style="color:#2F6F4E; font-size:14px;">
Separation of data into training, validation, and test sets following a strict chronological order.
</p>
</div>


In [39]:
train_end_year = 2018
val_end_year = 2021

# Split using time (NO shuffle, NO random seed)
train_df = df_model[df_model["year"] <= train_end_year].copy()
val_df = df_model[(df_model["year"] > train_end_year) & (df_model["year"] <= val_end_year)].copy()
test_df = df_model[df_model["year"] > val_end_year].copy()

# Basic checks
print("Train:", train_df.shape, "| Years:", train_df["year"].min(), "-", train_df["year"].max())
print("Val:  ", val_df.shape,   "| Years:", val_df["year"].min(), "-", val_df["year"].max())
print("Test: ", test_df.shape,  "| Years:", test_df["year"].min(), "-", test_df["year"].max())

Train: (432, 13) | Years: 2003 - 2018
Val:   (81, 13) | Years: 2019 - 2021
Test:  (54, 13) | Years: 2022 - 2023


Percentage-based splits were avoided due to the temporal nature of the forecasting task.

<div style="background-color:#EAF4EC; padding:16px; border-radius:10px;">
<h2 style="color:#2F6F4E; margin-bottom:5px; font-size:20px;">
Baseline Models
</div>


In [40]:
def eval_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return mae, rmse

In [41]:
# Baseline 1: Persistence (naive)
# y_hat(t) = y(t-1)
val_naive = val_df.copy()
test_naive = test_df.copy()

val_naive["y_pred"] = val_naive["lag_1"]
test_naive["y_pred"] = test_naive["lag_1"]

mae_val_naive, rmse_val_naive = eval_model(
    val_naive["value"],
    val_naive["y_pred"]
)

mae_test_naive, rmse_test_naive = eval_model(
    test_naive["value"],
    test_naive["y_pred"]
)

print("Naive")
print(f"Validation -> MAE: {mae_val_naive:.4f} | RMSE: {rmse_val_naive:.4f}")
print(f"Test       -> MAE: {mae_test_naive:.4f} | RMSE: {rmse_test_naive:.4f}")

Naive
Validation -> MAE: 0.0755 | RMSE: 0.1045
Test       -> MAE: 0.0572 | RMSE: 0.0699


<div style="background-color:#EAF4EC; padding:16px; border-radius:10px;">
<h2 style="color:#2F6F4E; margin-bottom:5px; font-size:20px;">
Random Forest
</h2>
</div>

In [159]:
features = [
    "lag_1",
    "lag_2",
    "lag_3",
    "rolling_mean_3",
    "rolling_std_3",
    "trend"
]

target = "value"
X_train = train_df[features]
y_train = train_df[target]

X_val = val_df[features]
y_val = val_df[target]

X_test = test_df[features]
y_test = test_df[target]

In [160]:
rf = RandomForestRegressor(
    n_estimators=600,
    max_depth=10,
    min_samples_leaf=5,
    min_samples_split=10,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)

val_pred = rf.predict(X_val)
test_pred = rf.predict(X_test)

mae_val, rmse_val = eval_model(y_val, val_pred)
mae_test, rmse_test = eval_model(y_test, test_pred)

print("Random Forest")
print(f"Validation -> MAE: {mae_val:.4f} | RMSE: {rmse_val:.4f}")
print(f"Test       -> MAE: {mae_test:.4f} | RMSE: {rmse_test:.4f}")

Random Forest
Validation -> MAE: 0.0843 | RMSE: 0.1162
Test       -> MAE: 0.0618 | RMSE: 0.0753


In [163]:
feat_imp = pd.Series(rf.feature_importances_, index=features).sort_values(ascending=False)
print("\nFeature importance:")
print(feat_imp)


Feature importance:
lag_1             0.674743
rolling_mean_3    0.287015
lag_2             0.025223
lag_3             0.011521
rolling_std_3     0.000903
trend             0.000595
dtype: float64


<div style="background-color:#EAF4EC; padding:16px; border-radius:10px;">
<h2 style="color:#2F6F4E; margin-bottom:5px; font-size:20px;">
XGBoost
</h2>
</div

In [164]:
xgb = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=3,
    min_child_weight=5,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_alpha=0.1,            
    reg_lambda=2.0,
    objective="reg:squarederror",
    random_state=42,
    n_jobs=-1
)

xgb.fit(X_train, y_train)

# ---- 4) Predict ----
val_pred = xgb.predict(X_val)
test_pred = xgb.predict(X_test)

mae_val, rmse_val = eval_model(y_val, val_pred)
mae_test, rmse_test = eval_model(y_test, test_pred)

print("XGBoost")
print(f"Validation -> MAE: {mae_val:.4f} | RMSE: {rmse_val:.4f}")
print(f"Test       -> MAE: {mae_test:.4f} | RMSE: {rmse_test:.4f}")

XGBoost
Validation -> MAE: 0.0871 | RMSE: 0.1181
Test       -> MAE: 0.0662 | RMSE: 0.0824


In [165]:
feat_imp = pd.Series(xgb.feature_importances_, index=features).sort_values(ascending=False)
print("\nFeature importance:")
print(feat_imp)


Feature importance:
lag_1             0.667663
rolling_mean_3    0.286917
lag_2             0.037019
lag_3             0.005243
trend             0.001645
rolling_std_3     0.001514
dtype: float32


<div style="background-color:#EAF4EC; padding:16px; border-radius:10px;">
<h2 style="color:#2F6F4E; margin-bottom:5px; font-size:20px;">
ANN
</h2>
</div>

In [166]:
ann = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("mlp", MLPRegressor(
        hidden_layer_sizes=(64, 32),
        activation="relu",
        solver="adam",
        alpha=1e-3,              # regularização (L2)
        learning_rate_init=1e-3,
        max_iter=2000,
        early_stopping=True,
        validation_fraction=0.2,  # dentro do TRAIN apenas
        n_iter_no_change=30,
        random_state=42
    ))
])

ann.fit(X_train, y_train)

val_pred = ann.predict(X_val)
test_pred = ann.predict(X_test)

mae_val, rmse_val = eval_model(y_val, val_pred)
mae_test, rmse_test = eval_model(y_test, test_pred)

print("ANN (MLPRegressor)")
print(f"Validation -> MAE: {mae_val:.4f} | RMSE: {rmse_val:.4f}")
print(f"Test       -> MAE: {mae_test:.4f} | RMSE: {rmse_test:.4f}")


ANN (MLPRegressor)
Validation -> MAE: 0.0850 | RMSE: 0.1146
Test       -> MAE: 0.0656 | RMSE: 0.0808


In [167]:
mlp = ann.named_steps["mlp"]
print("Iterations:", mlp.n_iter_)
print("Train loss:", mlp.loss_)


Iterations: 82
Train loss: 0.003992082196763724


<div style="background-color:#EAF4EC; padding:16px; border-radius:10px;">
<h2 style="color:#2F6F4E; margin-bottom:5px; font-size:20px;">
LSTM
</h2>
</div>

In [183]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from sklearn.preprocessing import StandardScaler

seq_cols = ["lag_3", "lag_2", "lag_1"]
static_cols = ["rolling_mean_3", "rolling_std_3", "trend"]
target = "value"

def make_lstm_arrays(df, fit_scaler=False, scaler=None):
    y = df[target].values.astype(np.float32)

    X_all = df[seq_cols + static_cols].values.astype(np.float32)

    if fit_scaler:
        scaler = StandardScaler()
        X_all = scaler.fit_transform(X_all)
    else:
        X_all = scaler.transform(X_all)

    # split
    X_seq = X_all[:, :len(seq_cols)]          # (n, 3)
    X_static = X_all[:, len(seq_cols):]       # (n, 3)

    # reshape sequence -> (n, 3, 1)
    X_seq = X_seq.reshape(-1, len(seq_cols), 1)

    # repeat static -> (n, 3, 3)
    X_static_rep = np.repeat(X_static[:, None, :], repeats=len(seq_cols), axis=1)

    # concat -> (n, 3, 4)
    X = np.concatenate([X_seq, X_static_rep], axis=2).astype(np.float32)
    return X, y, scaler

X_train_lstm, y_train_lstm, scaler = make_lstm_arrays(train_df, fit_scaler=True)
X_val_lstm, y_val_lstm, _ = make_lstm_arrays(val_df, fit_scaler=False, scaler=scaler)
X_test_lstm, y_test_lstm, _ = make_lstm_arrays(test_df, fit_scaler=False, scaler=scaler)

print(X_train_lstm.shape, X_val_lstm.shape, X_test_lstm.shape)


(16, 3, 4) (3, 3, 4) (2, 3, 4)


In [186]:
tf.random.set_seed(42)
np.random.seed(42)

model = models.Sequential([
    layers.Input(shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])),
    layers.LSTM(32),
    layers.Dropout(0.2),
    layers.Dense(16, activation="relu"),
    layers.Dense(1)
])

model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss="mse")

es = callbacks.EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True)

model.fit(
    X_train_lstm, y_train_lstm,
    validation_data=(X_val_lstm, y_val_lstm),
    epochs=300,
    batch_size=4,
    callbacks=[es],
    verbose=0
)


In [187]:
val_pred = model.predict(X_val_lstm).ravel()
test_pred = model.predict(X_test_lstm).ravel()

mae_val, rmse_val = eval_model(y_val_lstm, val_pred)
mae_test, rmse_test = eval_model(y_test_lstm, test_pred)

print("LSTM")
print(f"Validation -> MAE: {mae_val:.4f} | RMSE: {rmse_val:.4f}")
print(f"Test       -> MAE: {mae_test:.4f} | RMSE: {rmse_test:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
LSTM
Validation -> MAE: 0.0398 | RMSE: 0.0435
Test       -> MAE: 0.0334 | RMSE: 0.0378


<div style="background-color:#EAF4EC; padding:16px; border-radius:10px;">
<h2 style="color:#2F6F4E; margin-bottom:5px; font-size:20px;">
Hyperparamethrs Optimization using Optuna
</h2>
</div>

<div style="background-color:#EAF4EC; padding:16px; border-radius:10px;">
<h2 style="color:#2F6F4E; margin-bottom:5px; font-size:20px;">
XGBoost with Optuna
</h2>
</div>

In [192]:
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1500),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "max_depth": trial.suggest_int("max_depth", 2, 8),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 1.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 10.0, log=True),
        "random_state": 42,
        "objective": "reg:squarederror",
        "tree_method": "hist",
        "n_jobs": -1
    }

    model = XGBRegressor(**params)
    model.fit(X_train, y_train)

    val_pred = model.predict(X_val)
    rmse_val = np.sqrt(mean_squared_error(y_val, val_pred))

    return rmse_val  # queremos minimizar

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

print("Best RMSE (val):", study.best_value)
print("Best params:", study.best_params)


[I 2026-01-14 22:56:52,356] A new study created in memory with name: no-name-0825190e-c3c5-4f9c-8ee3-5df421c16537
[I 2026-01-14 22:56:53,007] Trial 0 finished with value: 0.11087376521900554 and parameters: {'n_estimators': 1276, 'learning_rate': 0.03807092978859271, 'max_depth': 6, 'subsample': 0.7402167983677895, 'colsample_bytree': 0.8494089572042748, 'min_child_weight': 3, 'reg_alpha': 0.002834552807003283, 'reg_lambda': 0.16949489693340036}. Best is trial 0 with value: 0.11087376521900554.
[I 2026-01-14 22:56:53,265] Trial 1 finished with value: 0.15073342751441693 and parameters: {'n_estimators': 899, 'learning_rate': 0.029781338528629023, 'max_depth': 6, 'subsample': 0.8284637354468424, 'colsample_bytree': 0.926736078902301, 'min_child_weight': 7, 'reg_alpha': 0.28127434296739956, 'reg_lambda': 0.02678192187852167}. Best is trial 0 with value: 0.11087376521900554.
[I 2026-01-14 22:56:53,746] Trial 2 finished with value: 0.11858320492929963 and parameters: {'n_estimators': 1310, 

Best RMSE (val): 0.016455043028741206
Best params: {'n_estimators': 200, 'learning_rate': 0.033393034115831825, 'max_depth': 5, 'subsample': 0.6347834203211173, 'colsample_bytree': 0.7404154713097343, 'min_child_weight': 1, 'reg_alpha': 0.00228880690149452, 'reg_lambda': 9.882964181152804}


In [193]:
from xgboost import XGBRegressor

best_params = {'n_estimators': 200,
               'learning_rate': 0.033393034115831825,
               'max_depth': 5,
               'subsample': 0.6347834203211173,
               'colsample_bytree': 0.7404154713097343,
               'min_child_weight': 1,
               'reg_alpha': 0.00228880690149452,
               'reg_lambda': 9.882964181152804}

xgb_best = XGBRegressor(
    **best_params,
    random_state=42,
    objective="reg:squarederror",
    tree_method="hist",
    n_jobs=-1
)

xgb_best.fit(X_train, y_train)

val_pred = xgb_best.predict(X_val)
test_pred = xgb_best.predict(X_test)

mae_val, rmse_val = eval_model(y_val, val_pred)
mae_test, rmse_test = eval_model(y_test, test_pred)

print("XGBoost (Optuna tuned)")
print(f"Validation -> MAE: {mae_val:.4f} | RMSE: {rmse_val:.4f}")
print(f"Test       -> MAE: {mae_test:.4f} | RMSE: {rmse_test:.4f}")


XGBoost (Optuna tuned)
Validation -> MAE: 0.0141 | RMSE: 0.0165
Test       -> MAE: 0.0355 | RMSE: 0.0386


In [194]:
print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)
print("Years train:", train_df["year"].min(), "-", train_df["year"].max())
print("Years val:  ", val_df["year"].min(), "-", val_df["year"].max())
print("Years test: ", test_df["year"].min(), "-", test_df["year"].max())
print("Countries train/val/test:",
      train_df["Country Code"].nunique(),
      val_df["Country Code"].nunique(),
      test_df["Country Code"].nunique())


Train: (16, 6) Val: (3, 6) Test: (2, 6)
Years train: 2003 - 2018
Years val:   2019 - 2021
Years test:  2022 - 2023
Countries train/val/test: 1 1 1


<div style="background-color:#EAF4EC; padding:16px; border-radius:10px;">
<h2 style="color:#2F6F4E; margin-bottom:5px; font-size:20px;">
Random Forest with Optuna
</h2>
</div>

In [197]:

def objective_rf(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1500),
        "max_depth": trial.suggest_int("max_depth", 3, 30),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", 0.5, 0.8, 1.0]),
        "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
        "random_state": 42,
        "n_jobs": -1
    }

    # se bootstrap=False, oob_score não faz sentido (e nem vamos usar oob_score)
    model = RandomForestRegressor(**params)
    model.fit(X_train, y_train)

    val_pred = model.predict(X_val)
    rmse_val = np.sqrt(mean_squared_error(y_val, val_pred))

    return rmse_val

study_rf = optuna.create_study(direction="minimize")
study_rf.optimize(objective_rf, n_trials=50)

print("RF Best RMSE (val):", study_rf.best_value)
print("RF Best params:", study_rf.best_params)


[I 2026-01-14 23:02:42,241] A new study created in memory with name: no-name-581e99f8-668d-44da-a4a2-9aa3ff7e3b6c
[I 2026-01-14 23:02:42,509] Trial 0 finished with value: 0.09550646248908781 and parameters: {'n_estimators': 737, 'max_depth': 27, 'min_samples_split': 3, 'min_samples_leaf': 7, 'max_features': 0.8, 'bootstrap': False}. Best is trial 0 with value: 0.09550646248908781.
[I 2026-01-14 23:02:42,973] Trial 1 finished with value: 0.0691152933591263 and parameters: {'n_estimators': 1404, 'max_depth': 4, 'min_samples_split': 3, 'min_samples_leaf': 6, 'max_features': 'sqrt', 'bootstrap': False}. Best is trial 1 with value: 0.0691152933591263.
[I 2026-01-14 23:02:43,248] Trial 2 finished with value: 0.29907365449803996 and parameters: {'n_estimators': 869, 'max_depth': 27, 'min_samples_split': 20, 'min_samples_leaf': 8, 'max_features': 0.5, 'bootstrap': False}. Best is trial 1 with value: 0.0691152933591263.
[I 2026-01-14 23:02:43,372] Trial 3 finished with value: 0.0398298962379035

RF Best RMSE (val): 0.022168005641861572
RF Best params: {'n_estimators': 375, 'max_depth': 12, 'min_samples_split': 8, 'min_samples_leaf': 2, 'max_features': 'log2', 'bootstrap': True}


In [198]:
best_params_rf = study_rf.best_params

rf_best = RandomForestRegressor(
    **best_params_rf,
    random_state=42,
    n_jobs=-1
)

rf_best.fit(X_train, y_train)

val_pred = rf_best.predict(X_val)
test_pred = rf_best.predict(X_test)

mae_val, rmse_val = eval_model(y_val, val_pred)
mae_test, rmse_test = eval_model(y_test, test_pred)

print("Random Forest (Optuna tuned)")
print(f"Validation -> MAE: {mae_val:.4f} | RMSE: {rmse_val:.4f}")
print(f"Test       -> MAE: {mae_test:.4f} | RMSE: {rmse_test:.4f}")


Random Forest (Optuna tuned)
Validation -> MAE: 0.0205 | RMSE: 0.0222
Test       -> MAE: 0.0430 | RMSE: 0.0514


| Indicator Code        | Nome (Indicator Name)                                                             | Medida / Unidade                                  |
| --------------------- | --------------------------------------------------------------------------------- | ------------------------------------------------- |
| **SM.POP.NETM**       | Net migration                                                                     | **N.º de pessoas** (migração líquida)             |
| **IP.JRN.ARTC.SC**    | Scientific and technical journal articles                                         | **N.º de artigos** (contagem anual)               |
| **IP.PAT.RESD**       | Patent applications, residents                                                    | **N.º de pedidos de patente** (residentes)        |
| **NY.GDP.MKTP.KD.ZG** | GDP growth (annual %)                                                             | **Percentagem (%)** crescimento anual do PIB real |
| **IT.NET.USER.ZS**    | Individuals using the Internet (% of population)                                  | **Percentagem (%)** da população                  |
| **SG.GEN.PARL.ZS**    | Proportion of seats held by women in national parliaments (%)                     | **Percentagem (%)** de lugares                    |
| **SL.TLF.CACT.FM.ZS** | Ratio of female to male labor force participation rate (%) (modeled ILO estimate) | **Percentagem (%)** (rácio female/male)           |
| **SE.ENR.PRSC.FM.ZS** | School enrollment, primary and secondary (gross), gender parity index (GPI)       | **Índice (GPI)** (paridade raparigas/rapazes)     |
| **RQ.EST**            | Regulatory Quality: Estimate                                                      | **Estimate (WGI score)** (~ -2.5 a 2.5)           |
| **GE.EST**            | Government Effectiveness: Estimate                                                | **Estimate (WGI score)** (~ -2.5 a 2.5)           |
| **CC.EST**            | Control of Corruption: Estimate                                                   | **Estimate (WGI score)** (~ -2.5 a 2.5)           |
| **PV.EST**            | Political Stability and Absence of Violence/Terrorism: Estimate                   | **Estimate (WGI score)** (~ -2.5 a 2.5)           |
| **RL.EST**            | Rule of Law: Estimate                                                             | **Estimate (WGI score)** (~ -2.5 a 2.5)           |
| **VA.EST**            | Voice and Accountability: Estimate                                                | **Estimate (WGI score)** (~ -2.5 a 2.5)           |
| **GB.XPD.RSDV.GD.ZS** | Research and development expenditure (% of GDP) | **Percentagem (%) do PIB** |
